In [76]:
from selenium import webdriver
from time import sleep
import datetime

from bs4 import BeautifulSoup
import requests
import base64

import os
import re
import shutil



In [114]:
#これでGoogleを開く
driver = webdriver.Chrome("C:\\Users\\solud\\OneDrive\\デスクトップ\\chromedriver")
driver.get("https://www.google.com/")
sleep(2)

In [115]:
#Googleの検索欄を指定している。
#検証を行う場合はseleniumで表示されたChromeで検証を行いながら行っていく必要がある
search_bar = driver.find_element_by_name("q")

In [116]:
#検索バーにキーワードを入れる
search_bar.send_keys("橋本環奈")#★★★

In [117]:
#これで検索を行う。
search_bar.submit()
sleep(1)

In [118]:
#画像画面に移動
img_btn = driver.find_element_by_xpath('//a[@class="q qs"]')
img_btn.click()

In [119]:
#画面のスクロールを行う。
try:
    #この中にはimageが重複している。
    all_images = []
    #スクロールする回数は10回
    for i in range(10):
        #ここで画面をスクロールしている。
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        #ここでBeautifulSoupに読み込んでいる。
        soup = BeautifulSoup(driver.page_source , "html.parser")
        
        #all_imagesに画像URLをappendする
        for image in soup.find_all("img"):
            try:
                url = image.get("data-src")

                if url is None:
                    url = image.get("src")

                if url is not None:
                    all_images.append(url)
            except:
                print("なんかのエラーが今起きてしまいました！！！")
                print()
        
        #今回は5回のスクロールで終了
        if i > 5:
            break
        sleep(2)           

except Exception:
    print("画面スクロール中にエラーが発生しました。")
    error_flag = True

In [120]:
#すべてのimgリンクのリスト
#リストの中の重複した分を取り除いていく。
all_images = list(dict.fromkeys(all_images))

In [121]:
#image_画像が何枚あるのかを確認。
len(all_images)

440

In [122]:
#すずちゃん用のダウンロードファイルを作成
os.makedirs("kanna_img_files")#★★★

In [123]:
#base64のものを保存していく関数
#urlには"data:image/jpeg;base64,"を取り除いたものをぶち込むようにする。
def base64_download(url , file_path):
    img = base64.b64decode(url.encode())
    with open(file_path , "wb") as f:
        f.write(img)
    


#httpのurlで渡された画像を保存していく。
def img_url_download(url , file_path):
    response = requests.get(url , stream = True)
    
    #ファイルに保存する、
    with open(file_path , 'wb') as file:
        shutil.copyfileobj(response.raw , file)

In [124]:
#画像データをファイルにぶち込んでいく！！

#ファイルのパス
path = r"C:\Users\solud\練習\kanna_img_files"#★★★

#base64のやつは最初に"data:image/jpeg;base64,"があるので取り除くようにする。
base64_string = "data:image/jpeg;base64,"

for index , image_url in enumerate(all_images):
    filename = "kanna_" + str(index) + ".jpg"#★★★
    file_path = os.path.join(path , filename)
    
    if len(re.findall(base64_string , image_url)) > 0:
        url = url.replace(base64_string , "")#接頭辞をurlからなくしている。
        base64_download(url , file_path)
    
    else:
        img_url_download(image_url , file_path)
        
    
    
print("終了！！")